In [16]:
import os
import pickle
import random
with open("./pickles/cdf_real2.pickle", "rb") as f:
    vlist = pickle.load(f)
vlist

['/home/od/Desktop/Dataset/CelebDF/Real/videos/id12_0000.mp4',
 '/home/od/Desktop/Dataset/CelebDF/Real/videos/00266.mp4',
 '/home/od/Desktop/Dataset/CelebDF/Real/videos/id39_0005.mp4',
 '/home/od/Desktop/Dataset/CelebDF/Real/videos/00258.mp4',
 '/home/od/Desktop/Dataset/CelebDF/Real/videos/id10_0007.mp4',
 '/home/od/Desktop/Dataset/CelebDF/Real/videos/id1_0001.mp4',
 '/home/od/Desktop/Dataset/CelebDF/Real/videos/00061.mp4',
 '/home/od/Desktop/Dataset/CelebDF/Real/videos/id22_0000.mp4']

In [17]:
from glob import glob
folders = ["cropped_faces", "cropped_faces_250", "landmarks", "semantic_videos", "cropped_faces(landmark)"]
files = []
for v in vlist:
    for folder in folders:
        # files.extend(glob(f"/home/od/Desktop/Dataset/DFDC/{folder}/{os.path.split(v)[-1][:-4]}*.*"))
      files.extend(glob(v.replace("videos",folder).replace(".mp4","*.*")))
files

['/home/od/Desktop/Dataset/CelebDF/Real/cropped_faces/id12_0000.avi',
 '/home/od/Desktop/Dataset/CelebDF/Real/cropped_faces_250/id12_0000.avi',
 '/home/od/Desktop/Dataset/CelebDF/Real/landmarks/id12_0000.npy',
 '/home/od/Desktop/Dataset/CelebDF/Real/semantic_videos/id12_0000_notate.avi',
 '/home/od/Desktop/Dataset/CelebDF/Real/semantic_videos/id12_0000.avi',
 '/home/od/Desktop/Dataset/CelebDF/Real/cropped_faces(landmark)/id12_0000.npy',
 '/home/od/Desktop/Dataset/CelebDF/Real/cropped_faces/00266.avi',
 '/home/od/Desktop/Dataset/CelebDF/Real/cropped_faces_250/00266.avi',
 '/home/od/Desktop/Dataset/CelebDF/Real/landmarks/00266.npy',
 '/home/od/Desktop/Dataset/CelebDF/Real/semantic_videos/00266_notate.avi',
 '/home/od/Desktop/Dataset/CelebDF/Real/semantic_videos/00266.avi',
 '/home/od/Desktop/Dataset/CelebDF/Real/cropped_faces(landmark)/00266.npy',
 '/home/od/Desktop/Dataset/CelebDF/Real/cropped_faces/id39_0005.avi',
 '/home/od/Desktop/Dataset/CelebDF/Real/cropped_faces_250/id39_0005.avi'

In [18]:
len(files),len(vlist)

(48, 8)

In [19]:
for file in files:
    os.remove(file)

In [ ]:
import cv2
import facer
import numpy as np
import torch
from tqdm import tqdm
from matplotlib import pyplot as plt
from torchvision.transforms.functional import normalize
from preprocessing.extract_faces import get_video_clip, save_video_lossless

# video_path = "/home/od/Desktop/Dataset/CelebDF/Fake/cropped_faces/id0_id1_0000.avi"
video_path = "/home/od/Desktop/Dataset/DFDC/videos/nwpjsjarju.mp4"
fps, frames = get_video_clip(video_path, stride=1)

In [ ]:
frame_indices = [i for i in range(0, 300, 1)]

In [ ]:
landmarks = np.load(f"/home/od/Desktop/Dataset/DFDC/test_lm/rfwtnztcsj.npy")

In [ ]:
landmarks.shape

In [ ]:
landmarks.reshape(-1,2).shape

In [ ]:
# import random
# i = random.randrange(0, len(frame_indices))
# print(i)
# plt.figure(figsize=(25, 15))
# plt.imshow(cv2.cvtColor(frames[frame_indices[i]], cv2.COLOR_BGR2RGB))
# plt.scatter(landmarks[i, :, 0], landmarks[i, :, 1], alpha=0.5, s=1)

In [ ]:
import cv2
import facer
import numpy as np
import torch
from tqdm import tqdm
from matplotlib import pyplot as plt
from torchvision.transforms.functional import normalize
from preprocessing.extract_faces import get_video_clip, save_video_lossless

# video_path = "/home/od/Desktop/Dataset/CelebDF/Fake/cropped_faces/id0_id1_0000.avi"
video_path = "/stock/FaceForensicC23/videos/NT/217_117.mp4"
fps, frames = get_video_clip(video_path, stride=1)

In [ ]:
class FaceData:
    def __init__(self, _lm, _bbox, _idx):
        self.ema_lm = _lm
        self.lm = [_lm]
        self.bbox = [_bbox]
        self.idx = [_idx]

    def last(self):
        return self.ema_lm

    def add(self, _lm, _bbox, _idx):
        self.ema_lm = self.ema_lm*0.5 +_lm * 0.5
        self.lm.append(_lm)
        self.bbox.append(_bbox)
        self.idx.append(_idx)

    def __len__(self):
        return len(self.lm)


In [ ]:
import pickle
with open("./pickles/217_117.pickle","rb") as f:
    frame_faces = pickle.load(f)

In [ ]:
face_dbs = []
for faces, index in zip(frame_faces, frame_indices):
    frame_landmarks = np.stack([face["landmarks"] for face in faces])
    matched_indices = []

    for face_data in face_dbs:

        lm_diff = np.sum(
            np.linalg.norm(frame_landmarks - face_data.last(), axis=-1),
            axis=1
        ) / frame_landmarks.shape[1]

        if (np.min(lm_diff) > 100):
            continue

        closest_idx = np.argmin(lm_diff)
        matched_indices.append(closest_idx)
        face_data.add(faces[closest_idx]["landmarks"], faces[closest_idx]["bbox"], index)

    for i, face in enumerate(faces):
        if i in matched_indices:
            continue
        else:
            _landmark = face['landmarks']
            _bbox = face['bbox']
            face_dbs.append(FaceData(_landmark, _bbox, index))

In [ ]:
print(len(face_dbs),len(face_dbs[0].lm),len(frames))

In [ ]:
len(sorted(face_dbs,key=lambda x: len(x),reverse=True)[0])

In [ ]:
[i for i in range(0,len(frames)) if not i in face_dbs[0].idx]

In [ ]:
import random
fid = 0
i = random.randrange(0, len(face_dbs[fid]))
i =210
landmarks = face_dbs[fid].lm[i]
plt.figure(figsize=(25, 15))
plt.imshow(cv2.cvtColor(frames[face_dbs[fid].idx[i]], cv2.COLOR_BGR2RGB))
plt.scatter(landmarks[:, 0], landmarks[:, 1], alpha=0.5, s=5)

In [ ]:
import random
idx = 211
faces = [face["landmarks"] for face in frame_faces[idx]]
print(len(faces))
landmarks =np.stack(faces).reshape((-1,2))
plt.figure(figsize=(25, 15))
plt.imshow(cv2.cvtColor(frames[idx], cv2.COLOR_BGR2RGB))
plt.scatter(landmarks[:, 0], landmarks[:, 1], alpha=0.5, s=5)